In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))
import plotly.graph_objects as go
import numpy as np
import pandas as pd
pd.options.display.max_columns = None
from pybaseball import batting_stats, pitching_stats, cache, playerid_lookup, statcast_batter, statcast_pitcher

cache.enable()
cache.config.cache_type='csv'
cache.config.save()


In [2]:
num_teams = 12
num_dollars = 260
player_split = .65
pitcher_split = 1 - player_split
tot_dollars = num_teams * num_dollars

drafted_by_pos = {
    'C':12,
    '1B':12,
    '2B':12,
    '3B':12,
    'SS':12,
    'OF':5*12,
    'MI':12,
    'CI':12,
    'DH':12*2, 
    'P':9
}

def load_data():
    h = pd.read_csv('data/2022-fangraphs-proj-h.csv')
    h['sorter'] = h['HR']+h['R']+h['RBI']+h['H']+h['SB']
    
    p = pd.read_csv('data/2022-fangraphs-proj-p.csv')
    val_h = pd.read_csv('data/2022-fangraphs-auction-calculator-h.csv')
    val_h.rename(columns={'PlayerId':'playerid', 'POS':'Pos'},inplace=True)
    val_p = pd.read_csv('data/2022-fangraphs-auction-calculator-p.csv')
    val_p.rename(columns={'PlayerId':'playerid', 'POS':'Pos'},inplace=True)
    
    h = h.merge(val_h[['playerid', 'Pos', 'Dollars']])
    h.drop(columns=['wOBA', 'CS', 'Fld', 'BsR', 'ADP'],inplace=True)
    h['Pos'] = h['Pos'].apply(lambda x: ', '.join(x.split('/')))
    h.sort_values('sorter', ascending=False, inplace=True)
    h.reset_index(drop=True)
    
    p = p.merge(val_p[['playerid', 'Pos', 'Dollars']])
    p.drop(columns=['ADP'],inplace=True)
    p['Sv+Hld'] = p['SV']+p['HLD']
    p['Pos'] = p['Pos'].apply(lambda x: ', '.join(x.split('/')))
    p['sorter'] = p['SO']+(p['Sv+Hld']*4)+p['W']
    p.sort_values('sorter', ascending=False, inplace=True)
    p.reset_index(drop=True)
    return h, p

def calc_z(x, stat):
    z = (x - drafted[stat].mean()) / drafted[stat].std()
    return z

def find_primary_pos(p):
    pos_list = p.split(', ')
    pos_hierarchy = ['C', '2B', '1B', 'OF', '3B', 'SS', 'DH', 'SP', 'RP', 'P']
    for i in pos_hierarchy:
        if i in pos_list:
            return i

In [3]:
def process_top_hitters():
    # Define two empty dicts
    pos_avg = {}
    pos_std = {}
    # Create Used field and set to False, for tracking which players are considered drafted
    h['Used'] = False
    # For each of these positions, define a mask to isolate the unused players who are eligible at that position
    for position in ['C', '2B', '1B', 'OF', '3B', 'SS']:
        mask = (h['Pos'].str.contains(position)) & (h['Used']==False)
        pos_avg[position], pos_std[position] = {}, {}
        
        # Rate stats first
        # Calculate the BA Z score. Because it is a rate, it takes a different formula: H - (AB * (lgH/lgAB))
        pos_index_list = h[mask].index[:drafted_by_pos[position]]
        h.loc[pos_index_list, 'BA'] = (h[mask]['H'] - (h[mask]['AB'] * (h[mask]['H'].sum()/h[mask]['AB'].sum())))
        
        # For each stat category, fill in the dictionaries with an average and standard deviation using the top N players
        # where N is established by the number of drafted players at that position by the league (eg 1B = 12, OF=60)
        for stat in ['PA', 'AB', 'BA', 'HR', 'RBI', 'R', 'SB']:
            pos_avg[position][stat] = round(h.loc[h[mask].index[:drafted_by_pos[position]], stat].mean(),1)
            pos_std[position][stat] = round(h.loc[h[mask].index[:drafted_by_pos[position]], stat].std(),1)
            # Using the player's stat projection, calculate their Z score among the top players
            for j in h[mask].index[:drafted_by_pos[position]]:
                h.loc[j, 'z'+stat] = (h.loc[j][stat] - pos_avg[position][stat]) / pos_std[position][stat]
        
        # Sum the 5 stat category Z scores
        h.loc[h[mask].index[:drafted_by_pos[position]], 'z'] = h['zR'] + h['zRBI'] + h['zHR'] + h['zBA'] + h['zSB']
        # Make the last player's Z score equal 0, then adjust the rest by that same amount
        if h.loc[h[mask].index[:drafted_by_pos[position]]].sort_values('z')['z'].iloc[0] < 0:
            h.loc[h[mask].index[:drafted_by_pos[position]], 'z'] += abs(h.loc[h[mask].index[:drafted_by_pos[position]]].sort_values('z')['z'].iloc[0])
        else:
            h.loc[h[mask].index[:drafted_by_pos[position]], 'z'] -= h.loc[h[mask].index[:drafted_by_pos[position]]].sort_values('z')['z'].iloc[0]
        # Assign the current position as the player's Primary_Pos
        h.loc[h[mask].index[:drafted_by_pos[position]], 'Primary_Pos'] = position
        #print(position+':\n',h.loc[h[mask].index[:drafted_by_pos[position]]]['Name'].unique())
        # Mark these players as Used so they do not get used in another position
        h.loc[h[mask].index[:drafted_by_pos[position]], 'Used'] = True

    # This is the same process as above except it does it for the MI and CI categories which means you have to find the 
    # top 12 middle/corner infielders available
    for position in ['MI', 'CI']:
        if position == 'MI':
            pos_avg[position], pos_std[position] = {}, {}
            mask = ((h['Pos'].str.contains('SS')) & (h['Used']==False)) | ((h['Pos'].str.contains('2B')) & (h['Used']==False))
            
            pos_index_list = h[mask].index[:drafted_by_pos[position]]
            h.loc[pos_index_list, 'BA'] = (h[mask]['H'] - (h[mask]['AB'] * (h[mask]['H'].sum()/h[mask]['AB'].sum())))
        
            for stat in ['PA', 'AB', 'BA', 'HR', 'RBI', 'R', 'SB']:
                pos_avg[position][stat] = round(h.loc[h[mask].index[:drafted_by_pos[position]], stat].mean(),1)
                pos_std[position][stat] = round(h.loc[h[mask].index[:drafted_by_pos[position]], stat].std(),1)
                for j in h[mask].index[:drafted_by_pos[position]]:
                    h.loc[j, 'z'+stat] = (h.loc[j][stat] - pos_avg[position][stat]) / pos_std[position][stat]

            h.loc[h[mask].index[:drafted_by_pos[position]], 'z'] = h['zR'] + h['zRBI'] + h['zHR'] + h['zBA'] + h['zSB']
            h.loc[h[mask].index[:drafted_by_pos[position]], 'z'] += abs(h.loc[h[mask].index[:drafted_by_pos[position]]].sort_values('z')['z'].iloc[0])
            h.loc[h[mask].index[:drafted_by_pos[position]], 'Primary_Pos'] = position
            #print(position+':\n',h.loc[h[mask].index[:12]]['Name'].unique())
            h.loc[h[mask].index[:drafted_by_pos[position]], 'Used'] = True

        elif position == 'CI':
            pos_avg[position], pos_std[position] = {}, {}
            mask = ((h['Pos'].str.contains('1B')) & (h['Used']==False)) | ((h['Pos'].str.contains('3B')) & (h['Used']==False))
            
            pos_index_list = h[mask].index[:drafted_by_pos[position]]
            h.loc[pos_index_list, 'BA'] = (h[mask]['H'] - (h[mask]['AB'] * (h[mask]['H'].sum()/h[mask]['AB'].sum())))
            
            for stat in ['PA', 'AB', 'BA', 'HR', 'RBI', 'R', 'SB']:
                pos_avg[position][stat] = round(h.loc[h[mask].index[:drafted_by_pos[position]], stat].mean(),1)
                pos_std[position][stat] = round(h.loc[h[mask].index[:drafted_by_pos[position]], stat].std(),1)
                for j in h[mask].index[:drafted_by_pos[position]]:
                    h.loc[j, 'z'+stat] = (h.loc[j][stat] - pos_avg[position][stat]) / pos_std[position][stat]

            h.loc[h[mask].index[:drafted_by_pos[position]], 'z'] = h['zR'] + h['zRBI'] + h['zHR'] + h['zBA'] + h['zSB']
            h.loc[h[mask].index[:drafted_by_pos[position]], 'z'] += abs(h.loc[h[mask].index[:drafted_by_pos[position]]].sort_values('z')['z'].iloc[0])
            h.loc[h[mask].index[:drafted_by_pos[position]], 'Primary_Pos'] = position
            #print(position+':\n',h.loc[h[mask].index[:12]]['Name'].unique())
            h.loc[h[mask].index[:drafted_by_pos[position]], 'Used'] = True

    # Same process again but uses all remaining hitters and takes the top 12. Expect to see the true DHs at the top
    pos_avg['DH'], pos_std['DH'] = {}, {}
    mask = (h['Used']==False)
    
    pos_index_list = h[mask].index[:drafted_by_pos[position]]
    h.loc[pos_index_list, 'BA'] = (h[mask]['H'] - (h[mask]['AB'] * (h[mask]['H'].sum()/h[mask]['AB'].sum())))
        
    for stat in ['PA', 'AB', 'BA', 'HR', 'RBI', 'R', 'SB']:
        pos_avg['DH'][stat] = round(h.loc[h[mask].index[:24], stat].mean(),1)
        pos_std['DH'][stat] = round(h.loc[h[mask].index[:24], stat].std(),1)
        for j in h[mask].index[:24]:
                h.loc[j, 'z'+stat] = (h.loc[j][stat] - pos_avg['DH'][stat]) / pos_std['DH'][stat]

    h.loc[h[mask].index[:24], 'z'] = h['zR'] + h['zRBI'] + h['zHR'] + h['zBA'] + h['zSB']
    h.loc[h[mask].index[:24], 'z'] += abs(h.loc[h[mask].index[:24]].sort_values('z')['z'].iloc[0])
    h.loc[h[mask].index[:24], 'Primary_Pos'] = 'DH'
    #print('DH:\n',h.loc[h[mask].index[:24]]['Name'].unique())
    #print('DH:\n',h.loc[h[mask].index[:24]].index)
    sub_mask = h.loc[h[mask].index[:24]].index
    h.loc[h[mask].index[:24], 'Used'] = True
    
    if len(h[h['Used']==True])!=14*num_teams:
        print('drafted list not right')
    return pos_avg, pos_std


In [4]:
def process_rem_hitters(pos_avg, pos_std):
    for position in ['C', '2B', '1B', 'OF', '3B', 'SS']:
        mask = (h['Used']==False) & (h['Primary_Pos']==position)
        h.loc[mask, 'BA'] = (h[mask]['H'] - (h[mask]['AB'] * (h[(h['Used']==True) & (h['Primary_Pos']==position)]['H'].sum()/h[(h['Used']==True) & (h['Primary_Pos']==position)]['AB'].sum())))
        for stat in ['PA', 'AB', 'BA', 'HR', 'RBI', 'R', 'SB']:
            h.loc[mask, 'z'+stat] = (h[stat] - pos_avg[position][stat]) / pos_std[position][stat]

    h.loc[h['Used']==False, 'z'] = h['zBA'] + h['zHR'] + h['zRBI'] + h['zR'] + h['zSB']
    return

In [5]:
h, p = load_data()
h['Primary_Pos'] = h['Pos'].apply(lambda x: find_primary_pos(x))
p['Primary_Pos'] = p['Pos'].apply(lambda x: find_primary_pos(x))
pos_avg, pos_std = process_top_hitters()
process_rem_hitters(pos_avg, pos_std)
tot_z = h[h['Used']==True]['z'].sum()
h['Value'] = (h['z'] / tot_z) * tot_dollars * player_split
h.sort_values('Value', ascending=False).head(10)
h['Owner'] = None
h['Paid'] = None

#### Next Steps  
<li>BA: 
<li>Do it again with pitchers


In [6]:
def process_pitchers():
    #(p['ER']*9) - (p['IP'] * ((lgERsum * 9)/(lgIPsum))) * -1
    p['xER'] = ((p['ER']*9) - (p['IP'] * (p['ER'].iloc[:108].sum()*9)/p['IP'].iloc[:108].sum())) * -1
    p['xWHIP'] = ((p['H']+p['BB']) - (p['IP'] * ((p['H'].iloc[:108].sum()+p['BB'].iloc[:108].sum()) / p['IP'].iloc[:108].sum()))) * -1
    p_avg = p.iloc[:108][['W', 'SO', 'Sv+Hld', 'xER', 'xWHIP']].mean()
    p_std = p.iloc[:108][['W', 'SO', 'Sv+Hld', 'xER', 'xWHIP']].std()
    for i in ['W', 'SO', 'Sv+Hld', 'xER', 'xWHIP']:
        p['z'+i] = p[i].apply(lambda x: (x - p_avg[i]) / p_std[i])
    p['z'] = p['zW'] + p['zSO'] + p['zSv+Hld'] + p['zxER'] + p['zxWHIP']
    p_adjustment_value = abs(p.sort_values('z', ascending=False).iloc[107]['z'])
    p['z'] = p['z'].apply(lambda x: x+p_adjustment_value)
    return

In [7]:
#(p['ER']*9) - (p['IP'] * ((lgERsum * 9)/(lgIPsum))) * -1
p['xER'] = ((p['ER']*9) - (p['IP'] * (p['ER'].iloc[:108].sum()*9)/p['IP'].iloc[:108].sum())) * -1

In [8]:
p['xWHIP'] = ((p['H']+p['BB']) - (p['IP'] * ((p['H'].iloc[:108].sum()+p['BB'].iloc[:108].sum()) / p['IP'].iloc[:108].sum()))) * -1

In [9]:
p_avg = p.iloc[:108][['W', 'SO', 'Sv+Hld', 'xER', 'xWHIP']].mean()
p_std = p.iloc[:108][['W', 'SO', 'Sv+Hld', 'xER', 'xWHIP']].std()

In [71]:
for i in ['W', 'SO', 'Sv+Hld', 'xER', 'xWHIP']:
    p['z'+i] = p[i].apply(lambda x: (x - p_avg[i]) / p_std[i])

In [72]:
p['z'] = p['zW'] + p['zSO'] + p['zSv+Hld'] + p['zxER'] + p['zxWHIP']

In [73]:
p_adjustment_value = abs(p.sort_values('z', ascending=False).iloc[107]['z'])
p['z'] = p['z'].apply(lambda x: x+p_adjustment_value)

In [74]:
process_pitchers()

In [75]:
total_z = h[h['z']>0]['z'].sum()+p[p['z']>0]['z'].sum()
print('Z sum:',total_z)
print('Dollars available:',tot_dollars)
value_mult = tot_dollars/total_z
print('Dollars per Z unit:',value_mult)
print('% of Z from pitchers:',p[p['z']>0]['z'].sum()/total_z)

Z sum: 805.5072638055515
Dollars available: 3120
Dollars per Z unit: 3.8733356484705324
% of Z from pitchers: 0.29490781778445857


In [76]:
p['Value'] = p['z'] * value_mult

In [77]:
h['Value'] = h['z'] * value_mult

In [78]:
h.sort_values('Value', ascending=False).head(20)

,Name,Team,G,PA,AB,H,2B,3B,HR,R,RBI,BB,SO,HBP,SB,AVG,OBP,SLG,OPS,WAR,playerid,sorter,Pos,Dollars,Primary_Pos,Used,BA,zPA,zAB,zBA,zHR,zRBI,zR,zSB,z,Value,Owner,Paid
3,Vladimir Guerrero Jr.,TOR,154,665,575,178,31,2,44,109,122,77,102,6,4,0.309,0.394,0.598,0.992,6.0,19611,457,"1B, DH",40.252202,1B,True,33.335606,0.469136,0.066225,2.389664,1.854545,2.071429,2.022727,0.476190,14.063823,54.473907,None,None
1,Fernando Tatis Jr.,SDP,151,651,564,158,30,2,44,112,103,72,161,7,25,0.281,0.366,0.575,0.941,6.7,19709,442,"SS, OF",40.784208,OF,True,17.161705,1.085635,0.795666,1.153108,2.100000,1.459854,2.669643,2.208333,13.641913,52.839708,None,None
0,Juan Soto,WSN,154,665,519,161,29,2,37,112,106,135,96,5,12,0.310,0.453,0.586,1.039,7.2,20123,428,OF,37.932103,OF,True,31.398803,1.472376,-0.597523,2.548902,1.225000,1.678832,2.669643,0.402778,12.576129,48.711571,None,None
77,Salvador Perez,KCR,149,601,560,146,26,1,36,79,99,26,147,10,1,0.260,0.302,0.504,0.806,2.7,7304,361,"C, DH",25.055800,C,True,13.626102,1.846395,2.221122,1.449278,2.566667,2.485507,1.950495,-0.750000,11.792357,45.675756,None,None
17,Bo Bichette,TOR,152,658,601,176,36,2,28,96,94,43,121,6,17,0.292,0.343,0.496,0.839,4.7,19612,411,"SS, DH",28.080286,SS,True,22.819346,0.762542,1.044218,1.601935,0.900000,1.367089,1.632911,0.900000,11.621147,45.012602,None,None
45,Shohei Ohtani,LAA,147,637,537,138,24,5,38,103,96,86,176,5,23,0.257,0.363,0.533,0.895,3.5,19755,398,"P, OF, DH",30.421044,OF,True,3.903963,0.698895,-0.040248,-0.146670,1.350000,0.948905,1.866071,1.930556,9.999836,38.732723,None,None
5,Aaron Judge,NYY,152,658,562,154,25,1,40,103,107,83,170,5,5,0.274,0.370,0.537,0.907,5.7,15640,409,"OF, DH",28.931068,OF,True,13.661131,1.279006,0.733746,0.809915,1.600000,1.751825,1.866071,-0.569444,9.509341,36.832870,None,None
15,Bryce Harper,PHI,154,665,540,145,32,1,35,102,101,113,150,7,12,0.269,0.398,0.531,0.929,4.8,11579,395,OF,27.485023,OF,True,10.154824,1.472376,0.052632,0.466159,0.975000,1.313869,1.776786,0.402778,8.985566,34.804112,None,None
2,Mike Trout,LAA,147,637,511,138,25,3,37,100,101,109,147,11,7,0.271,0.406,0.546,0.952,6.1,10155,383,OF,25.942635,OF,True,10.396509,0.698895,-0.845201,0.489854,1.225000,1.313869,1.598214,-0.291667,8.386245,32.482740,None,None
19,Freddie Freeman,NaN,159,686,581,169,33,2,30,102,96,88,111,8,6,0.290,0.389,0.511,0.900,4.6,5361,403,1B,25.954566,1B,True,22.826065,1.765432,0.463576,1.349115,-0.690909,-0.250000,1.227273,1.428571,8.313318,32.200270,None,None


In [55]:
h[['Name', 'z', 'Value']]

,Name,z,Value
3,Vladimir Guerrero Jr.,14.063823,54.473907
1,Fernando Tatis Jr.,13.641913,52.839708
0,Juan Soto,12.576129,48.711571
13,Rafael Devers,7.643909,29.607424
17,Bo Bichette,11.621147,45.012602
...,...,...,...
585,Jacob Amaya,-29.166122,-112.970179
569,Andy Young,-27.078466,-104.883986
564,Brett Sullivan,-16.095308,-62.342529
563,Chadwick Tromp,-16.095308,-62.342529


In [79]:
print(p[p['z']>0].shape)
p[p['z']>0]['z'].sum()

(107, 35)


237.5503893784254

In [57]:
print(h[h['z']>0].shape)
h[h['z']>0]['z'].sum()

(147, 36)


567.9568744271261

In [80]:
107/(107+568)

0.15851851851851853

In [81]:
p.iloc[:108][p['Primary_Pos']=='RP'].shape

<ipython-input-81-dbec7db4bfd9>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p.iloc[:108][p['Primary_Pos']=='RP'].shape


(35, 35)

In [82]:
p[(p['Primary_Pos']=='RP')]

,Name,Team,W,L,SV,HLD,ERA,GS,G,IP,H,ER,HR,SO,BB,WHIP,K/9,BB/9,FIP,WAR,playerid,Pos,Dollars,Sv+Hld,sorter,Primary_Pos,xER,xWHIP,zW,zSO,zSv+Hld,zxER,zxWHIP,z,Value
95,Edwin Diaz,NYM,4,3,33,2,2.92,0,67,67.0,47,22,7,98,25,1.07,13.16,3.36,2.88,1.8,14710,RP,15.221505,35,242,RP,58.016296,10.860915,-1.245155,-0.980565,1.944845,0.879035,0.778649,3.250084,12.588667
99,Aroldis Chapman,NYY,4,2,32,2,2.92,0,64,64.0,42,21,6,98,32,1.16,13.71,4.48,3.05,1.7,10233,RP,13.301703,34,238,RP,55.552880,5.150725,-1.245155,-0.980565,1.869990,0.841711,0.369270,2.728525,10.568494
64,Liam Hendriks,CHW,4,3,29,2,2.85,0,68,68.0,48,22,8,100,17,0.96,13.19,2.22,2.67,2.3,3548,RP,16.676213,31,228,RP,61.837435,19.097645,-1.245155,-0.943896,1.645425,0.936931,1.369163,3.635743,14.082454
71,Josh Hader,MIL,4,3,26,2,2.72,0,67,67.0,42,20,7,110,27,1.02,14.79,3.63,2.66,2.1,14212,RP,15.897362,28,226,RP,76.016296,13.860915,-1.245155,-0.760549,1.420860,1.151762,0.993727,3.433920,13.300727
115,Raisel Iglesias,LAA,4,3,31,2,3.25,0,68,68.0,54,25,9,87,19,1.08,11.58,2.58,3.28,1.5,17130,RP,12.668511,33,223,RP,34.837435,11.097645,-1.245155,-1.182246,1.795135,0.527840,0.795621,2.564470,9.933052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553,Ryne Harper,NaN,2,2,0,0,4.71,0,38,38.0,39,20,6,32,14,1.40,7.61,3.38,4.77,0.0,12680,RP,-8.925814,0,34,RP,-34.796727,-6.004257,-1.775635,-2.190653,-0.675082,-0.527223,-0.430462,-3.725780,-14.431195
524,Alex Claudio,NaN,2,2,0,0,4.18,0,40,40.0,42,19,4,31,15,1.43,6.90,3.41,4.34,0.0,12890,RP,-8.373443,0,33,RP,-18.154450,-7.530797,-1.775635,-2.208987,-0.675082,-0.275067,-0.539904,-3.601400,-13.949433
551,Robert Gsellman,NaN,2,2,0,0,4.65,0,39,39.0,41,20,6,31,14,1.43,7.16,3.33,4.70,0.0,13696,RP,-8.959704,0,33,RP,-30.975589,-6.767527,-1.775635,-2.208987,-0.675082,-0.469327,-0.485183,-3.740939,-14.489913
521,Matt Peacock,ARI,2,2,0,1,4.33,0,36,36.0,40,17,4,26,13,1.45,6.49,3.15,4.27,0.0,20339,RP,-8.670840,1,32,RP,-15.439005,-8.477717,-1.775635,-2.300661,-0.600227,-0.233924,-0.607791,-3.644963,-14.118165


In [25]:
from sqlalchemy import text
pid = '19611'
price = 38
owner = 'Harvey'
t = text("UPDATE hitting SET Owner='"+owner+"', Paid="+str(price)+" WHERE playerid='"+pid+"'")
result = conn.execute(t)
result

In [26]:
pd.read_sql('hitting', conn)

,index,Name,Team,G,PA,AB,H,2B,3B,HR,R,RBI,BB,SO,HBP,SB,AVG,OBP,SLG,OPS,WAR,playerid,sorter,Pos,Dollars,Primary_Pos,Used,BA,zPA,zAB,zBA,zHR,zRBI,zR,zSB,z,Value,Owner,Paid
0,3,Vladimir Guerrero Jr.,TOR,154,665,575,178,31,2,44,109,122,77,102,6,4,0.309,0.394,0.598,0.992,6.0,19611,457,"1B, DH",40.252202,1B,True,33.335606,0.469136,0.066225,2.389664,1.854545,2.071429,2.022727,0.476190,14.063823,54.473907,Harvey,38
1,1,Fernando Tatis Jr.,SDP,151,651,564,158,30,2,44,112,103,72,161,7,25,0.281,0.366,0.575,0.941,6.7,19709,442,"SS, OF",40.784208,OF,True,17.161705,1.085635,0.795666,1.153108,2.100000,1.459854,2.669643,2.208333,13.641913,52.839708,None,None
2,0,Juan Soto,WSN,154,665,519,161,29,2,37,112,106,135,96,5,12,0.310,0.453,0.586,1.039,7.2,20123,428,OF,37.932103,OF,True,31.398803,1.472376,-0.597523,2.548902,1.225000,1.678832,2.669643,0.402778,12.576129,48.711571,None,None
3,13,Rafael Devers,BOS,154,665,593,165,37,2,37,97,108,58,136,7,6,0.278,0.346,0.534,0.880,4.8,17350,413,3B,27.907182,3B,True,16.010347,0.900344,1.043333,0.901833,1.260274,1.455882,1.447368,-0.278689,7.643909,29.607424,None,None
4,17,Bo Bichette,TOR,152,658,601,176,36,2,28,96,94,43,121,6,17,0.292,0.343,0.496,0.839,4.7,19612,411,"SS, DH",28.080286,SS,True,22.819346,0.762542,1.044218,1.601935,0.900000,1.367089,1.632911,0.900000,11.621147,45.012602,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623,585,Jacob Amaya,LAD,2,7,6,1,0,0,0,1,1,1,2,0,0,0.228,0.291,0.368,0.659,0.0,sa3005081,3,SS,-34.996832,SS,False,-0.600818,-21.010033,-19.193878,-0.740082,-6.100000,-10.405063,-10.392405,-1.528571,-29.166122,-112.970179,None,None
624,569,Andy Young,ARI,2,7,6,1,0,0,0,1,1,0,2,0,0,0.221,0.291,0.399,0.690,0.0,19521,3,2B,-35.270584,2B,False,-0.617596,-38.389222,-28.686869,-0.922454,-4.921569,-9.728395,-9.911111,-1.594937,-27.078466,-104.883986,None,None
625,564,Brett Sullivan,MIL,2,6,6,1,0,0,0,1,1,0,1,0,0,0.231,0.285,0.358,0.643,0.0,sa857232,3,C,-25.916788,C,False,-0.499706,-7.479624,-6.920792,-1.166612,-3.433333,-4.615942,-5.772277,-1.107143,-16.095308,-62.342529,None,None
626,563,Chadwick Tromp,ATL,2,6,6,1,0,0,0,1,1,0,2,0,0,0.225,0.275,0.367,0.641,0.0,16953,3,C,-25.916788,C,False,-0.499706,-7.479624,-6.920792,-1.166612,-3.433333,-4.615942,-5.772277,-1.107143,-16.095308,-62.342529,None,None


https://leportella.com/sqlalchemy-tutorial/

In [5]:
from sqlalchemy import MetaData, text, Column, Integer, String, ForeignKey, Table, create_engine, Float, Boolean
from sqlalchemy.orm import relationship, backref, sessionmaker
from sqlalchemy.ext.declarative import declarative_base

meta = MetaData()
engine = create_engine('sqlite:///fantasy_data.db', echo=False)
Session = sessionmaker(bind=engine)
session = Session()
Base = declarative_base()

In [120]:
#Creates a table on the DB
meta = MetaData()
drafted = Table('drafted', meta,
                    Column('playerid', String, primary_key=True),
                    Column('owner', String(25)),
                    Column('cost', Integer),
                    Column('pos', Integer),
)
meta.create_all(engine)

In [6]:
conn = engine.connect()

In [7]:
#Creates a table on the DB
meta = MetaData()
hitters = Table('hitting', meta,
                Column('playerid', String, primary_key=True),
                #Column('name', String(50)),
                #Column('team', String(20)),
                #Column('pa', Integer),
                #Column('ab', Integer),
                #Column('ba', Float),
                #Column('h', Integer),
                #Column('hr', Integer),
                #Column('r', Integer),
                #Column('rbi', Integer),
                #Column('sb', Integer),
                Column('Paid', Integer),
                Column('Owner', String(25)),
                Column('Used', Boolean)
)
meta.create_all(engine)

In [97]:
#conn.execute(hitters.delete().where(hitters.c.Used==None))

In [78]:
conn.execute(hitters.update().values(Paid=15, Owner='Harvey').where(hitters.c.playerid=='15172'))
conn.execute(hitters.update().values(Paid=1, Owner='Harvey').where(hitters.c.playerid=='16375'))

In [8]:
h = pd.read_sql('hitting', engine)
owners_df = h.groupby('Owner').agg({'Name':'count', 'Paid':'sum', 'z':'sum', 'H':'sum', 'AB':'sum', 'HR':'sum', 'R':'sum', 'RBI':'sum', 'SB':'sum'}).reset_index()
owners_df.rename(columns={'Name':'Drafted'},inplace=True)
owners_df['$/unit'] = owners_df['Paid']/owners_df['z']
owners_df['$ Left'] = 260 - owners_df['Paid']
owners_df['BA'] = owners_df['H']/owners_df['AB']
owners_df['Rank'] = 0
for i in ['BA', 'HR', 'R', 'RBI', 'SB']:
    owners_df['Rank'] += owners_df[i].rank()
owners_df

,Owner,Drafted,Paid,z,H,AB,HR,R,RBI,SB,$/unit,$ Left,BA,Rank
0,Avg Joes,8,125.0,26.586737,1119,4297,187,644,622,104,4.701592,135.0,0.260414,36.5
1,Brewbirds,9,138.0,34.367205,1244,4759,242,782,735,60,4.015456,122.0,0.261399,40.0
2,Charmer,10,172.0,52.559155,1419,5490,295,832,878,75,3.272503,88.0,0.258470,42.0
3,Dirty Birds,7,83.0,3.190019,876,3450,158,509,504,40,26.018657,177.0,0.253913,8.5
4,Harvey,6,95.0,36.613419,877,3188,156,509,492,48,2.594677,165.0,0.275094,19.5
5,Lil Trump,8,111.0,27.237012,1122,4162,193,622,636,47,4.075337,149.0,0.269582,35.0
6,Lima Time,14,163.0,59.284860,1950,7380,371,1151,1145,110,2.749437,97.0,0.264228,55.0
7,Midnight,6,117.0,31.378819,847,3253,184,516,536,61,3.728630,143.0,0.260375,21.0
8,Moms Cookin,7,143.0,32.580367,978,3856,210,587,607,94,4.389146,117.0,0.253631,30.0
9,Roid Ragers,13,185.0,24.972017,1818,6838,298,1023,980,157,7.408292,75.0,0.265867,54.0


In [9]:
h[h['Owner']=='Lima Time'][['Name', 'Primary_Pos', 'Pos', 'Timestamp']].sort_values("Timestamp")

,Name,Primary_Pos,Pos,Timestamp
58,Robert Witt,SS,SS,2022-01-13 17:11:52.082550
1,Fernando Tatis Jr.,OF,"SS, OF",2022-01-13 19:56:29.806066
35,Marcus Semien,2B,"2B, SS",2022-01-13 20:04:02.166412
90,Yuli Gurriel,CI,1B,2022-01-13 21:38:40.792696
137,Michael Brantley,OF,"OF, DH",2022-01-13 21:39:20.844020
236,Sean Murphy,C,"C, DH",2022-01-13 21:39:53.215636
71,Rhys Hoskins,CI,1B,2022-01-13 21:40:21.470886
52,Alex Bregman,3B,3B,2022-01-13 21:41:49.406985
150,Mark Canha,DH,"OF, DH",2022-01-13 21:43:09.643134
61,Ian Happ,2B,"2B, OF",2022-01-13 21:43:37.805731


In [10]:
def check_roster_pos(roster, name, team_name, pos, eligible):
    eligible_at = eligible.split(', ')
    eligibility = []
    for p in eligible.split(', '):
        if p=='C':
            eligibility.extend(['C'])
        if p=='1B':
            eligibility.extend(['1B', 'CI'])
        if p=='2B':
            eligibility.extend(['2B', 'MI'])
        if p=='3B':
            eligibility.extend(['3B', 'CI'])
        if p=='SS':
            eligibility.extend(['SS', 'MI'])
        if p=='OF':
            eligibility.extend(['OF1', 'OF2', 'OF3', 'OF4', 'OF5'])
        
    eligibility = list(dict.fromkeys(eligibility))
    pos_list = eligibility+['DH1', 'DH2']
    for p in pos_list:
        if roster.loc[p, team_name]==0:
            roster.loc[p, team_name] = name
            return p
    
    return pos_list

In [11]:
roster = pd.DataFrame(index=['C', '1B', '2B', '3B', 'SS', 'MI', 'CI', 'OF1', 'OF2', 'OF3', 'OF4', 'OF5', 'DH1', 'DH2'], data=np.zeros((14,12)), columns=owners_df.Owner.tolist())

In [12]:
for tm in owners_df.Owner.to_list():
    for i, row in h[h['Owner']==tm][['Name', 'Owner', 'Primary_Pos', 'Pos', 'Timestamp']].sort_values("Timestamp").iterrows():
        check_roster_pos(roster, h.loc[i]['Name'], h.loc[i]['Owner'], h.loc[i]['Primary_Pos'], h.loc[i]['Pos'])

In [40]:
roster

,Avg Joes,Brewbirds,Charmer,Dirty Birds,Harvey,Lil Trump,Lima Time,Midnight,Moms Cookin,Roid Ragers,Trouble,Wu-Tang
C,Danny Jansen,Yasmani Grandal,Salvador Perez,Mike Zunino,Will Smith,Keibert Ruiz,Sean Murphy,0.0,0.0,0.0,Adley Rutschman,0.0
1B,Jose Abreu,Joey Votto,Kyle Schwarber,Paul Goldschmidt,Vladimir Guerrero Jr.,Josh Bell,Yuli Gurriel,Pete Alonso,Matt Olson,Trey Mancini,0.0,Nathaniel Lowe
2B,Whit Merrifield,Trea Turner,Javier Baez,0.0,0.0,Ketel Marte,Marcus Semien,Ozzie Albies,Jazz Chisholm Jr.,Mookie Betts,Gleyber Torres,Luis Urias
3B,Manny Machado,Anthony Rendon,Rafael Devers,0.0,0.0,0.0,Alex Bregman,Eugenio Suarez,Matt Chapman,Yandy Diaz,Wander Franco,0.0
SS,Nicky Lopez,0.0,Tim Anderson,Xander Bogaerts,Corey Seager,Carlos Correa,Robert Witt,0.0,0.0,Bo Bichette,0.0,0.0
MI,0.0,Ty France,Chris Taylor,0.0,Willy Adames,0.0,Fernando Tatis Jr.,0.0,0.0,Trevor Story,0.0,0.0
CI,0.0,Freddie Freeman,Anthony Rizzo,Eduardo Escobar,0.0,Kris Bryant,Rhys Hoskins,0.0,Jose Ramirez,Austin Riley,0.0,0.0
OF1,Shohei Ohtani,Joey Gallo,Aaron Judge,George Springer,Alex Verdugo,Bryan Reynolds,Michael Brantley,Bryce Harper,Starling Marte,Ronald Acuna Jr.,Juan Soto,Mike Trout
OF2,Teoscar Hernandez,Jesse Winker,Cody Bellinger,Dylan Carlson,Cedric Mullins II,Byron Buxton,Mark Canha,Luis Robert,Tyler O'Neill,Randy Arozarena,Giancarlo Stanton,Christian Yelich
OF3,Robbie Grossman,Trent Grisham,Michael Conforto,Victor Robles,0.0,Andrew Benintendi,Ian Happ,Ramon Laureano,Eloy Jimenez,Kyle Lewis,Charlie Blackmon,Hunter Renfroe


In [78]:
print('# of Players Yet to be Drafted:',(12*23) - owners_df.Drafted.sum())
print('$ still available to be spent:', 3120 - owners_df.Paid.sum())

# of Players Yet to be Drafted: 174
$ still available to be spent: 1584.0


In [70]:
print('Initial $ per z rate:',3120/h[h['z']>=0]['z'].sum()*.65)
print('Current $ per z rate:',owners_df.Paid.sum() / owners_df.z.sum())
#print((3120-h['Paid'].sum())/h[(h['z']>0) & (h['Paid']>0)]['z'])

Initial $ per z rate: 3.570693641212737
Current $ per z rate: 3.966182460363122


In [13]:
import json
j = h.set_index('Name').to_json(orient='index')
j = json.loads(j)
j['Joey Gallo']

{'index': 41,
 'Team': 'NYY',
 'G': 147,
 'PA': 637,
 'AB': 522,
 'H': 109,
 '2B': 18,
 '3B': 1,
 'HR': 41,
 'R': 91,
 'RBI': 99,
 'BB': 102,
 'SO': 218,
 'HBP': 7,
 'SB': 4,
 'AVG': 0.208,
 'OBP': 0.342,
 'SLG': 0.48,
 'OPS': 0.822,
 'WAR': 3.6,
 'playerid': '14128',
 'sorter': 344,
 'Pos': 'OF, DH',
 'Dollars': 14.2436686165,
 'Primary_Pos': 'OF',
 'Used': True,
 'BA': -21.3503371714,
 'zPA': 0.6988950276,
 'zAB': -0.5046439628,
 'zBA': -2.6225820756,
 'zHR': 1.725,
 'zRBI': 1.1678832117,
 'zR': 0.7946428571,
 'zSB': -0.7083333333,
 'z': 4.4075851569,
 'Value': 17.0720567119,
 'Owner': 'Brewbirds',
 'Paid': 15.0,
 'Timestamp': 1642197483808}

In [46]:
h.sort_values('z', ascending=False)

,index,Name,Team,G,PA,AB,H,2B,3B,HR,R,RBI,BB,SO,HBP,SB,AVG,OBP,SLG,OPS,WAR,playerid,sorter,Pos,Dollars,Primary_Pos,Used,BA,zPA,zAB,zBA,zHR,zRBI,zR,zSB,z,Value,Owner,Paid,Timestamp
0,3,Vladimir Guerrero Jr.,TOR,154,665,575,178,31,2,44,109,122,77,102,6,4,0.309,0.394,0.598,0.992,6.0,19611,457,"1B, DH",40.252202,1B,True,33.335606,0.469136,0.066225,2.389664,1.854545,2.071429,2.022727,0.476190,14.063823,54.473907,Harvey,24.0,2022-01-13 19:56:12.110798
1,1,Fernando Tatis Jr.,SDP,151,651,564,158,30,2,44,112,103,72,161,7,25,0.281,0.366,0.575,0.941,6.7,19709,442,"SS, OF",40.784208,OF,True,17.161705,1.085635,0.795666,1.153108,2.100000,1.459854,2.669643,2.208333,13.641913,52.839708,Lima Time,36.0,2022-01-13 19:56:29.806066
2,0,Juan Soto,WSN,154,665,519,161,29,2,37,112,106,135,96,5,12,0.310,0.453,0.586,1.039,7.2,20123,428,OF,37.932103,OF,True,31.398803,1.472376,-0.597523,2.548902,1.225000,1.678832,2.669643,0.402778,12.576129,48.711571,Trouble,39.0,2022-01-13 19:57:48.386218
39,77,Salvador Perez,KCR,149,601,560,146,26,1,36,79,99,26,147,10,1,0.260,0.302,0.504,0.806,2.7,7304,361,"C, DH",25.055800,C,True,13.626102,1.846395,2.221122,1.449278,2.566667,2.485507,1.950495,-0.750000,11.792357,45.675756,Charmer,21.0,2022-01-13 22:13:48.338069
4,17,Bo Bichette,TOR,152,658,601,176,36,2,28,96,94,43,121,6,17,0.292,0.343,0.496,0.839,4.7,19612,411,"SS, DH",28.080286,SS,True,22.819346,0.762542,1.044218,1.601935,0.900000,1.367089,1.632911,0.900000,11.621147,45.012602,Roid Ragers,30.0,2022-01-13 20:01:42.395693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,214,Eugenio Suarez,CIN,123,532,464,104,19,1,27,65,77,55,153,7,2,0.224,0.314,0.444,0.758,1.3,12552,275,"3B, SS",1.000000,DH,True,NaN,-1.069620,-1.199262,NaN,1.306452,1.321839,-0.671429,-1.196429,NaN,NaN,Midnight,6.0,2022-01-13 21:51:38.241228
159,118,Nico Hoerner,CHC,131,567,507,141,29,2,8,56,57,45,85,8,13,0.278,0.344,0.391,0.735,2.2,21479,275,"2B, SS, OF",-0.793976,DH,True,NaN,0.037975,0.387454,NaN,-1.758065,-0.977011,-1.957143,0.767857,NaN,NaN,None,NaN,NaT
350,412,Khris Davis,None,53,231,207,48,8,1,12,29,33,19,63,3,1,0.231,0.301,0.458,0.758,0.2,9112,123,DH,-18.514938,DH,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaT
466,423,Jacob Nottingham,BAL,36,141,127,26,5,0,5,15,15,10,46,2,2,0.202,0.270,0.365,0.635,0.2,16448,63,DH,-27.785540,DH,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaT


In [14]:
mask = (h['Owner'].isna()) & (h['Primary_Pos']=='1B')
h[mask][['Name', 'z', 'Dollars', 'Pos']].head(10)

,Name,z,Dollars,Pos
31,Jared Walsh,4.162144,17.504832,"1B, OF"
46,Ryan Mountcastle,3.448506,14.869896,"1B, OF, DH"
162,Alec Bohm,-10.336413,-1.009455,"1B, 3B"
165,Seth Brown,-9.652087,-1.844754,"1B, OF, DH"
168,Jesus Aguilar,-10.345003,1.246184,1B
176,Andrew Vaughn,-10.337913,-0.161136,"1B, OF, DH"
181,Miguel Cabrera,-12.321601,-3.075281,"1B, DH"
182,Bobby Bradley,-11.138588,-2.503977,"1B, DH"
188,LaMonte Wade Jr.,-10.196542,-2.913432,"1B, OF"
192,Gavin Sheets,-10.787844,-0.492015,"1B, OF, DH"


In [53]:
h['Paid'].sum()

1536.0

In [48]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=h[mask].sort_values('z', ascending=False)['Name'].head(10),
        y=h[mask]['z'].head(10),
    )
)
fig.show()

In [35]:
h[single_player][['zBA', 'zR', 'zRBI', 'zHR', 'zSB']].values.tolist()[0]

[2.5489022252324927,
 2.6696428571428577,
 1.6788321167883213,
 1.225,
 0.4027777777777778]

In [43]:
single_player = (h['Name']=='Robert Witt')

fig = go.Figure(data=go.Scatterpolar(
  r=h[single_player][['zBA', 'zR', 'zRBI', 'zHR', 'zSB']].values.tolist()[0],
  theta=['zBA', 'zR', 'zRBI', 'zHR', 'zSB'],
  fill='toself',
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True
    ),
  ),
  showlegend=False
)

fig.show()

In [72]:
#conn.execute('ALTER TABLE hitting ADD COLUMN Paid Integer')

In [86]:
#conn.execute('ALTER TABLE hitting DROP COLUMN Paid2')

In [102]:
#conn.execute('ALTER TABLE hitting ADD COLUMN Timestamp DATETIME')

In [75]:
conn.close()